# 데이터 인덱스 생성

In [1]:
import elasticsearch
print(f"설치된 elasticsearch 버전: {elasticsearch.__version__}")

설치된 elasticsearch 버전: (8, 15, 0)


In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

es.info()

2025-06-04 16:33:59,346 - INFO - GET http://localhost:9200/ [status:200 duration:0.018s]


ObjectApiResponse({'name': '04a4333d7186', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gUF90JznTHaeaQw8tnJrXg', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
from elasticsearch import Elasticsearch
import csv, json
import logging
from typing import Dict, Any, List, Optional
from datetime import datetime

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class KoreanContentElasticsearch:    
    def __init__(self, host: str = "localhost", port: int = 9200):
        """
        Elasticsearch 클라이언트 초기화
        
        Args:
            host: Elasticsearch 호스트 주소
            port: Elasticsearch 포트 번호
        """
        try:
            # 클라이언트 설정
            client_url = f"http://{host}:{port}"
            
            self.es = Elasticsearch(client_url)
            
            # 연결 테스트
            self._test_connection()
            
        except Exception as e:
            logger.error(f"Elasticsearch 연결 실패: {e}")
            raise
    
    def _test_connection(self) -> None:
        """연결 상태 확인"""
        try:
            info = self.es.info()
            logger.info(f"Elasticsearch 연결 성공!")
            logger.info(f"클러스터: {info['cluster_name']}")
            logger.info(f"버전: {info['version']['number']}")
            print(f"Elasticsearch 연결 성공!")
            print(f"클러스터: {info['cluster_name']}")
            print(f"버전: {info['version']['number']}")
        except Exception as e:
            logger.error(f"연결 테스트 실패: {e}")
            raise
    
    def create_korean_content_index(self, index_name: str) -> bool:
        """
        한국어 콘텐츠를 위한 인덱스 생성 (nori_tokenizer 사용)
        
        Args:
            index_name: 생성할 인덱스 이름
            
        Returns:
            생성 성공 여부
        """
        try:
            # 인덱스가 이미 존재하는지 확인
            if self.es.indices.exists(index=index_name):
                logger.warning(f"인덱스 '{index_name}'이 이미 존재합니다.")
                print(f"경고: 인덱스 '{index_name}'이 이미 존재합니다.")
                return True
            
            # 한국어 분석을 위한 설정
            index_settings = {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "analysis": {
                    "tokenizer": {
                        "nori_user_dict": {
                            "type": "nori_tokenizer",
                            "decompound_mode": "mixed",
                            "discard_punctuation": "true"
                        }
                    },
                    "analyzer": {
                        "korean_analyzer": {
                            "type": "custom",
                            "tokenizer": "nori_user_dict",
                            "filter": [
                                "nori_part_of_speech",
                                "nori_readingform",
                                "lowercase"
                            ]
                        },
                        "korean_search_analyzer": {
                            "type": "custom", 
                            "tokenizer": "nori_user_dict",
                            "filter": [
                                "nori_part_of_speech",
                                "lowercase"
                            ]
                        }
                    },
                    "filter": {
                        "nori_part_of_speech": {
                            "type": "nori_part_of_speech",
                            "stoptags": [
                                "E", "IC", "J", "MAG", "MAJ", "MM", 
                                "SP", "SSC", "SSO", "SC", "SE", "XPN", "XSA", "XSN", "XSV"
                            ]
                        }
                    }
                }
            }
            
            # 필드 매핑 정의
            field_mappings = {
                "HEX(id)": {
                    "type": "keyword"
                },
                "name": {
                    "type": "text", 
                    "analyzer": "korean_analyzer",
                    "search_analyzer": "korean_search_analyzer",
                    "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                    }
                },
                "category": {
                    "type": "text",
                    "analyzer": "korean_analyzer",
                    "search_analyzer": "korean_search_analyzer", 
                    "fields": {
                        "keyword": {
                            "type": "keyword"
                        }
                    }
                },
                "subCategory": {
                    "type": "text",
                    "analyzer": "korean_analyzer",
                    "search_analyzer": "korean_search_analyzer",
                    "fields": {
                        "keyword": {
                            "type": "keyword"
                        }
                    }
                },
                "address": {
                    "type": "text",
                    "analyzer": "korean_analyzer",
                    "search_analyzer": "korean_search_analyzer",
                    "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                    }
                },
                "content": {
                    "type": "text",
                    "analyzer": "korean_analyzer",
                    "search_analyzer": "korean_search_analyzer"
                }
            }
            
            # 인덱스 생성 요청 바디
            index_body = {
                "settings": index_settings,
                "mappings": {
                    "properties": field_mappings
                }
            }
            
            # 인덱스 생성
            logger.info(f"인덱스 '{index_name}' 생성 중...")
            print(f"인덱스 '{index_name}' 생성 중...")
            
            response = self.es.indices.create(index=index_name, body=index_body)
            
            if response.get('acknowledged', False):
                logger.info(f"인덱스 '{index_name}' 생성 완료")
                print(f"인덱스 '{index_name}' 생성 완료")
                print("매핑 정보:")
                print("- name: text (한국어 분석)")
                print("- category: text (한국어 분석)")  
                print("- subCategory: text (한국어 분석)")
                print("- address: text (한국어 분석)")
                print("- content: text (한국어 분석)")
                print("- timestamp: date (자동 생성)")
                print(f"사용 분석기: nori_tokenizer 기반 korean_analyzer")
                return True
            else:
                logger.error(f"인덱스 생성 응답 확인 실패")
                print(f"오류: 인덱스 생성 응답 확인 실패")
                return False
                
        except Exception as e:
            logger.error(f"인덱스 생성 실패: {e}")
            print(f"오류: 인덱스 생성 실패 - {e}")
            return False
    
    def insert_data_from_csv(self, index_name: str, csv_file_path: str, id_field: str = None) -> bool:
        """
        JSON 파일에서 데이터를 읽어서 벌크 삽입
        
        Args:
            index_name: 대상 인덱스 이름
            json_file_path: JSON 파일 경로
            id_field: 문서 ID로 사용할 필드명 (없으면 자동 생성)
            
        Returns:
            삽입 성공 여부
        """
        try:
            logger.info(f"CSV 파일 읽는 중: {csv_file_path}")
            print(f"CSV 파일 읽는 중: {csv_file_path}")
            
            # CSV 파일 읽기
            data = []
            with open(csv_file_path, "r", encoding="utf-8") as f:
                csv_reader = csv.DictReader(f)
                for row in csv_reader:
                    data.append(row)
            
            logger.info(f"읽어온 데이터 개수: {len(data)}")
            print(f"읽어온 데이터 개수: {len(data)}")
            
            # 필수 필드 확인
            required_fields = ["HEX(id)", "name", "category", "sub_category", "address", "content"]
            processed_data = []
            
            for i, doc in enumerate(data):
                # 필수 필드 존재 확인
                missing_fields = [field for field in required_fields if field not in doc]
                if missing_fields:
                    logger.warning(f"문서 {i+1}에서 누락된 필드: {missing_fields}")
                    print(f"경고: 문서 {i+1}에서 누락된 필드: {missing_fields}")
                    continue
                
                processed_data.append(doc)
            
            if not processed_data:
                logger.error("삽입할 유효한 데이터가 없습니다.")
                print("오류: 삽입할 유효한 데이터가 없습니다.")
                return False
            
            logger.info(f"유효한 데이터 개수: {len(processed_data)}")
            print(f"유효한 데이터 개수: {len(processed_data)}")
            
            # 벌크 삽입 준비
            bulk_body = []
            for i, doc in enumerate(processed_data):
                # 문서 ID 결정
                doc_id = doc.get(id_field) if id_field else i + 1
                
                # 벌크 작업 정의
                bulk_body.append({
                    "index": {
                        "_index": index_name,
                        "_id": doc_id
                    }
                })
                bulk_body.append(doc)
            
            logger.info("벌크 삽입 실행 중...")
            print("벌크 삽입 실행 중...")
            
            # 벌크 삽입 실행
            response = self.es.bulk(body=bulk_body)
            
            # 결과 출력
            logger.info(f"벌크 삽입 완료 - 성공: {success_count}개")
            print(f"벌크 삽입 완료 - 성공: {success_count}개")
            
            if errors:
                logger.warning(f"삽입 실패: {len(errors)}개")
                print(f"경고: 삽입 실패 {len(errors)}개")
                
                # 처음 3개 에러만 상세 출력
                for i, error in enumerate(errors[:3]):
                    logger.warning(f"에러 {i+1}: {error}")
                    print(f"에러 {i+1}: {error}")
                
                if len(errors) > 3:
                    logger.warning(f"추가로 {len(errors)-3}개의 에러가 더 있습니다.")
                    print(f"추가로 {len(errors)-3}개의 에러가 더 있습니다.")
        except FileNotFoundError:
            logger.error(f"파일을 찾을 수 없습니다: {json_file_path}")
            print(f"오류: 파일을 찾을 수 없습니다 - {json_file_path}")
            return False
        except json.JSONDecodeError as e:
            logger.error(f"JSON 파싱 오류: {e}")
            print(f"오류: JSON 파싱 실패 - {e}")
            return False
        except Exception as e:
            logger.error(f"데이터 삽입 실패: {e}")
            print(f"오류: 데이터 삽입 실패 - {e}")
            return False


In [21]:
# 사용 예시
if __name__ == "__main__":
    # 한국어 콘텐츠 Elasticsearch 클라이언트 초기화
    print("한국어 콘텐츠 Elasticsearch 클라이언트 초기화 중...")
    korean_es = KoreanContentElasticsearch(host="localhost", port=9200)
    
    # 인덱스 생성
    print("\n=== 한국어 콘텐츠 인덱스 생성 ===")
    index_name = "place_data_v3"
    success = korean_es.create_korean_content_index(index_name)
    
    if success:
        print(f"인덱스 '{index_name}' 생성 성공")
        
        # json 파일 삽입
        json_file_path = "../place.csv"
        korean_es.insert_data_from_csv(index_name, json_file_path)
        
    else:
        print("인덱스 생성 실패")


2025-06-04 17:36:15,813 - INFO - GET http://localhost:9200/ [status:200 duration:0.004s]
2025-06-04 17:36:15,814 - INFO - Elasticsearch 연결 성공!
2025-06-04 17:36:15,815 - INFO - 클러스터: docker-cluster
2025-06-04 17:36:15,816 - INFO - 버전: 8.15.0
2025-06-04 17:36:15,820 - INFO - HEAD http://localhost:9200/place_data_v3 [status:200 duration:0.002s]
2025-06-04 17:36:15,822 - WARNING - 인덱스 'place_data_v3'이 이미 존재합니다.
2025-06-04 17:36:15,823 - INFO - CSV 파일 읽는 중: ../place.csv


2025-06-04 17:36:15,895 - INFO - 읽어온 데이터 개수: 4185
2025-06-04 17:36:15,896 - WARNING - 문서 1에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,897 - WARNING - 문서 2에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,898 - WARNING - 문서 3에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,899 - WARNING - 문서 4에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,901 - WARNING - 문서 5에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,902 - WARNING - 문서 6에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,903 - WARNING - 문서 7에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,906 - WARNING - 문서 8에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,907 - WARNING - 문서 9에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,909 - WARNING - 문서 10에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,910 - WARNING - 문서 11에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,911 - WARNING - 문서 12에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,912 - WARNING - 문서 13에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,913 - WARNING - 문서 14에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:15,915 - WARNING - 문서 15에서 누락된 필드: ['HEX(id)']

한국어 콘텐츠 Elasticsearch 클라이언트 초기화 중...
Elasticsearch 연결 성공!
클러스터: docker-cluster
버전: 8.15.0

=== 한국어 콘텐츠 인덱스 생성 ===
경고: 인덱스 'place_data_v3'이 이미 존재합니다.
인덱스 'place_data_v3' 생성 성공
CSV 파일 읽는 중: ../place.csv
읽어온 데이터 개수: 4185
경고: 문서 1에서 누락된 필드: ['HEX(id)']
경고: 문서 2에서 누락된 필드: ['HEX(id)']
경고: 문서 3에서 누락된 필드: ['HEX(id)']
경고: 문서 4에서 누락된 필드: ['HEX(id)']
경고: 문서 5에서 누락된 필드: ['HEX(id)']
경고: 문서 6에서 누락된 필드: ['HEX(id)']
경고: 문서 7에서 누락된 필드: ['HEX(id)']
경고: 문서 8에서 누락된 필드: ['HEX(id)']
경고: 문서 9에서 누락된 필드: ['HEX(id)']
경고: 문서 10에서 누락된 필드: ['HEX(id)']
경고: 문서 11에서 누락된 필드: ['HEX(id)']
경고: 문서 12에서 누락된 필드: ['HEX(id)']
경고: 문서 13에서 누락된 필드: ['HEX(id)']
경고: 문서 14에서 누락된 필드: ['HEX(id)']
경고: 문서 15에서 누락된 필드: ['HEX(id)']
경고: 문서 16에서 누락된 필드: ['HEX(id)']
경고: 문서 17에서 누락된 필드: ['HEX(id)']
경고: 문서 18에서 누락된 필드: ['HEX(id)']
경고: 문서 19에서 누락된 필드: ['HEX(id)']
경고: 문서 20에서 누락된 필드: ['HEX(id)']
경고: 문서 21에서 누락된 필드: ['HEX(id)']
경고: 문서 22에서 누락된 필드: ['HEX(id)']
경고: 문서 23에서 누락된 필드: ['HEX(id)']
경고: 문서 24에서 누락된 필드: ['HEX(id)']
경고: 문서 25에서 누락된 필드: ['H

2025-06-04 17:36:16,018 - WARNING - 문서 77에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,027 - WARNING - 문서 78에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,029 - WARNING - 문서 79에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,031 - WARNING - 문서 80에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,033 - WARNING - 문서 81에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,035 - WARNING - 문서 82에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,037 - WARNING - 문서 83에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,039 - WARNING - 문서 84에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,041 - WARNING - 문서 85에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,049 - WARNING - 문서 86에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,050 - WARNING - 문서 87에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,052 - WARNING - 문서 88에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,060 - WARNING - 문서 89에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,061 - WARNING - 문서 90에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,062 - WARNING - 문서 91에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:16,064 - WARNING - 문서 9

경고: 문서 76에서 누락된 필드: ['HEX(id)']
경고: 문서 77에서 누락된 필드: ['HEX(id)']
경고: 문서 78에서 누락된 필드: ['HEX(id)']
경고: 문서 79에서 누락된 필드: ['HEX(id)']
경고: 문서 80에서 누락된 필드: ['HEX(id)']
경고: 문서 81에서 누락된 필드: ['HEX(id)']
경고: 문서 82에서 누락된 필드: ['HEX(id)']
경고: 문서 83에서 누락된 필드: ['HEX(id)']
경고: 문서 84에서 누락된 필드: ['HEX(id)']
경고: 문서 85에서 누락된 필드: ['HEX(id)']
경고: 문서 86에서 누락된 필드: ['HEX(id)']
경고: 문서 87에서 누락된 필드: ['HEX(id)']
경고: 문서 88에서 누락된 필드: ['HEX(id)']
경고: 문서 89에서 누락된 필드: ['HEX(id)']
경고: 문서 90에서 누락된 필드: ['HEX(id)']
경고: 문서 91에서 누락된 필드: ['HEX(id)']
경고: 문서 92에서 누락된 필드: ['HEX(id)']
경고: 문서 93에서 누락된 필드: ['HEX(id)']
경고: 문서 94에서 누락된 필드: ['HEX(id)']
경고: 문서 95에서 누락된 필드: ['HEX(id)']
경고: 문서 96에서 누락된 필드: ['HEX(id)']
경고: 문서 97에서 누락된 필드: ['HEX(id)']
경고: 문서 98에서 누락된 필드: ['HEX(id)']
경고: 문서 99에서 누락된 필드: ['HEX(id)']
경고: 문서 100에서 누락된 필드: ['HEX(id)']
경고: 문서 101에서 누락된 필드: ['HEX(id)']
경고: 문서 102에서 누락된 필드: ['HEX(id)']
경고: 문서 103에서 누락된 필드: ['HEX(id)']
경고: 문서 104에서 누락된 필드: ['HEX(id)']
경고: 문서 105에서 누락된 필드: ['HEX(id)']
경고: 문서 106에서 누락된 필드: ['HEX(id)']
경

...

경고: 문서 3995에서 누락된 필드: ['HEX(id)']
경고: 문서 3996에서 누락된 필드: ['HEX(id)']
경고: 문서 3997에서 누락된 필드: ['HEX(id)']
경고: 문서 3998에서 누락된 필드: ['HEX(id)']
경고: 문서 3999에서 누락된 필드: ['HEX(id)']
경고: 문서 4000에서 누락된 필드: ['HEX(id)']
경고: 문서 4001에서 누락된 필드: ['HEX(id)']
경고: 문서 4002에서 누락된 필드: ['HEX(id)']
경고: 문서 4003에서 누락된 필드: ['HEX(id)']
경고: 문서 4004에서 누락된 필드: ['HEX(id)']
경고: 문서 4005에서 누락된 필드: ['HEX(id)']
경고: 문서 4006에서 누락된 필드: ['HEX(id)']
경고: 문서 4007에서 누락된 필드: ['HEX(id)']
경고: 문서 4008에서 누락된 필드: ['HEX(id)']
경고: 문서 4009에서 누락된 필드: ['HEX(id)']
경고: 문서 4010에서 누락된 필드: ['HEX(id)']
경고: 문서 4011에서 누락된 필드: ['HEX(id)']
경고: 문서 4012에서 누락된 필드: ['HEX(id)']
경고: 문서 4013에서 누락된 필드: ['HEX(id)']
경고: 문서 4014에서 누락된 필드: ['HEX(id)']
경고: 문서 4015에서 누락된 필드: ['HEX(id)']
경고: 문서 4016에서 누락된 필드: ['HEX(id)']
경고: 문서 4017에서 누락된 필드: ['HEX(id)']
경고: 문서 4018에서 누락된 필드: ['HEX(id)']
경고: 문서 4019에서 누락된 필드: ['HEX(id)']
경고: 문서 4020에서 누락된 필드: ['HEX(id)']
경고: 문서 4021에서 누락된 필드: ['HEX(id)']
경고: 문서 4022에서 누락된 필드: ['HEX(id)']
경고: 문서 4023에서 누락된 필드: ['HEX(id)']
경고: 문서 4024에서 

2025-06-04 17:36:24,189 - WARNING - 문서 4068에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,191 - WARNING - 문서 4069에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,193 - WARNING - 문서 4070에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,195 - WARNING - 문서 4071에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,197 - WARNING - 문서 4072에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,199 - WARNING - 문서 4073에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,201 - WARNING - 문서 4074에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,203 - WARNING - 문서 4075에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,204 - WARNING - 문서 4076에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,207 - WARNING - 문서 4077에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,208 - WARNING - 문서 4078에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,209 - WARNING - 문서 4079에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,210 - WARNING - 문서 4080에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,222 - WARNING - 문서 4081에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,224 - WARNING - 문서 4082에서 누락된 필드: ['HEX(id)']
2025-06-04

경고: 문서 4067에서 누락된 필드: ['HEX(id)']
경고: 문서 4068에서 누락된 필드: ['HEX(id)']
경고: 문서 4069에서 누락된 필드: ['HEX(id)']
경고: 문서 4070에서 누락된 필드: ['HEX(id)']
경고: 문서 4071에서 누락된 필드: ['HEX(id)']
경고: 문서 4072에서 누락된 필드: ['HEX(id)']
경고: 문서 4073에서 누락된 필드: ['HEX(id)']
경고: 문서 4074에서 누락된 필드: ['HEX(id)']
경고: 문서 4075에서 누락된 필드: ['HEX(id)']
경고: 문서 4076에서 누락된 필드: ['HEX(id)']
경고: 문서 4077에서 누락된 필드: ['HEX(id)']
경고: 문서 4078에서 누락된 필드: ['HEX(id)']
경고: 문서 4079에서 누락된 필드: ['HEX(id)']
경고: 문서 4080에서 누락된 필드: ['HEX(id)']
경고: 문서 4081에서 누락된 필드: ['HEX(id)']
경고: 문서 4082에서 누락된 필드: ['HEX(id)']
경고: 문서 4083에서 누락된 필드: ['HEX(id)']
경고: 문서 4084에서 누락된 필드: ['HEX(id)']
경고: 문서 4085에서 누락된 필드: ['HEX(id)']
경고: 문서 4086에서 누락된 필드: ['HEX(id)']
경고: 문서 4087에서 누락된 필드: ['HEX(id)']
경고: 문서 4088에서 누락된 필드: ['HEX(id)']
경고: 문서 4089에서 누락된 필드: ['HEX(id)']
경고: 문서 4090에서 누락된 필드: ['HEX(id)']
경고: 문서 4091에서 누락된 필드: ['HEX(id)']
경고: 문서 4092에서 누락된 필드: ['HEX(id)']
경고: 문서 4093에서 누락된 필드: ['HEX(id)']
경고: 문서 4094에서 누락된 필드: ['HEX(id)']
경고: 문서 4095에서 누락된 필드: ['HEX(id)']
경고: 문서 4096에서 

2025-06-04 17:36:24,393 - WARNING - 문서 4171에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,395 - WARNING - 문서 4172에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,396 - WARNING - 문서 4173에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,397 - WARNING - 문서 4174에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,399 - WARNING - 문서 4175에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,400 - WARNING - 문서 4176에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,401 - WARNING - 문서 4177에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,403 - WARNING - 문서 4178에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,404 - WARNING - 문서 4179에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,417 - WARNING - 문서 4180에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,419 - WARNING - 문서 4181에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,422 - WARNING - 문서 4182에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,423 - WARNING - 문서 4183에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,425 - WARNING - 문서 4184에서 누락된 필드: ['HEX(id)']
2025-06-04 17:36:24,426 - WARNING - 문서 4185에서 누락된 필드: ['HEX(id)']
2025-06-04

경고: 문서 4171에서 누락된 필드: ['HEX(id)']
경고: 문서 4172에서 누락된 필드: ['HEX(id)']
경고: 문서 4173에서 누락된 필드: ['HEX(id)']
경고: 문서 4174에서 누락된 필드: ['HEX(id)']
경고: 문서 4175에서 누락된 필드: ['HEX(id)']
경고: 문서 4176에서 누락된 필드: ['HEX(id)']
경고: 문서 4177에서 누락된 필드: ['HEX(id)']
경고: 문서 4178에서 누락된 필드: ['HEX(id)']
경고: 문서 4179에서 누락된 필드: ['HEX(id)']
경고: 문서 4180에서 누락된 필드: ['HEX(id)']
경고: 문서 4181에서 누락된 필드: ['HEX(id)']
경고: 문서 4182에서 누락된 필드: ['HEX(id)']
경고: 문서 4183에서 누락된 필드: ['HEX(id)']
경고: 문서 4184에서 누락된 필드: ['HEX(id)']
경고: 문서 4185에서 누락된 필드: ['HEX(id)']
오류: 삽입할 유효한 데이터가 없습니다.
